In [1]:
!pip install minsearch

In [36]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [37]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [1]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [2]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [3]:
from openai import OpenAI
import openai
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()


def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [4]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [12]:
rag("how do I run Docker")

'To run Docker, you need to use the `docker run` command in a terminal. Here’s a basic example for running a PostgreSQL container:\n\n```\ndocker run -it \\\n-e POSTGRES_USER="root" \\\n-e POSTGRES_PASSWORD="root" \\\n-e POSTGRES_DB="ny_taxi" \\\n-v <your path>:/var/lib/postgresql/data \\\n-p 5432:5432 \\\npostgres:13\n```\n\nMake sure to replace `<your path>` with the appropriate path on your system.\n\nIf you\'re using Windows, ensure that you have the Docker engine set up correctly, either using Hyper-V (for Pro versions) or WSL2 (for Home versions). Additionally, if you face issues related to permission or needing elevated privileges, consult the Docker for Windows documentation for troubleshooting.'

# Part 1: Agentic RAG

In [16]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [41]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return context.strip()

In [13]:
import json

def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)
    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(answer)

    if answer['action'] == 'SEARCH':
        print('need to perform search...')
        search_results = search(question)
        context = build_context(search_results)
        
        prompt = prompt_template.format(question=question, context=context)
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(answer)

    return answer

In [44]:
agentic_rag_v1("Can I still join the course?")

{'action': 'SEARCH', 'reasoning': 'The context is empty, and I need to check if there are specific enrollment details in our FAQ database regarding joining the course.'}
need to perform search...
{'action': 'ANSWER', 'answer': "Yes, you can still join the course after it has started. Even if you haven't registered, you are eligible to submit the homework assignments. Just remember that there will be deadlines for the final projects, so plan accordingly to avoid rushing at the end.", 'source': 'CONTEXT'}


{'action': 'ANSWER',
 'answer': "Yes, you can still join the course after it has started. Even if you haven't registered, you are eligible to submit the homework assignments. Just remember that there will be deadlines for the final projects, so plan accordingly to avoid rushing at the end.",
 'source': 'CONTEXT'}

# Part 2: Agentic Search

In [75]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [45]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [62]:
question = "How do I do well on module 1"
max_iterations = 3
iteration_number = 0
search_queries = []
search_results = []
previous_actions = []

In [63]:

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [64]:
answer_json = llm(prompt)

In [65]:
answer = json.loads(answer_json)

In [74]:
previous_actions.append(answer)

In [66]:
keywords = answer['keywords']

In [67]:
keywords

['module 1 study tips', 'how to succeed in module 1', 'module 1 resources']

In [68]:
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [72]:
search_results = dedup(search_results)

In [73]:
len(search_results)

8

In [76]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []


iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

In [78]:
print(answer)

{'action': 'ANSWER', 'answer': 'To be successful in Module 1: Docker and Terraform, here are some tips:\n\n1. **Understand the Basics**: Make sure you have a good grasp of Docker and Terraform fundamentals. Familiarize yourself with concepts like images, containers, modules, and providers.\n\n2. **Hands-On Practice**: Regularly practice with Docker and Terraform in real projects or exercises. Setting up local environments and deploying applications will reinforce your learning.\n\n3. **Follow Documentation**: Always refer to the official documentation for Docker and Terraform. It will help you understand best practices and troubleshoot issues effectively.\n\n4. **Stay Updated**: Docker and Terraform receive frequent updates. Keeping your tools updated and following any changes in best practices is important for your success.\n\n5. **Utilize Community Resources**: Engage with community forums or groups where you can ask questions and share your experiences with peers.\n\n6. **Experiment

# Function Calling ("Tool Use")

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [6]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [75]:
def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [59]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"how to do well in module 1"}', call_id='call_Uu2BSkx0AMDXFVMjRTynmogz', name='search', type='function_call', id='fc_687b407c03e881a185b9b2f568801d420a7631bef30c3e97', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"tips for success in module 1"}', call_id='call_D5E0pqusLpQKwC1Rv2350y7d', name='search', type='function_call', id='fc_687b407c38a481a191278aab049c96dc0a7631bef30c3e97', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"module 1 best practices"}', call_id='call_B1DestjSypKwau7Obb0NQFdn', name='search', type='function_call', id='fc_687b407c5e3081a1be30ce40b0f5234d0a7631bef30c3e97', status='completed')]

In [60]:
calls = response.output

In [71]:
for call in calls:
    result = do_call(call)
    chat_messages.append(call)
    chat_messages.append(result)

In [72]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)

In [ ]:
for entry in response.output:
    chat_messages.append(entry)
    print(entry.type)

    if entry.type == 'function_call':      
        result = do_call(entry)
        chat_messages.append(result)
    elif entry.type == 'message':
        print(entry.text) 

## Putting all together (Cleaner)

In [76]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [77]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break

 hoow to run docker in windows 10


function_call: ResponseFunctionToolCall(arguments='{"query":"run Docker in Windows 10"}', call_id='call_kGeohlE3vtTeGqy8nNl3o11c', name='search', type='function_call', id='fc_687b4548aa7081928441c0a4ef25cfb1077f84ad0a6d4f14', status='completed')

To run Docker on Windows 10, follow these steps:

### For Windows 10 Pro Users:
1. **Enable Hyper-V**: 
   - Search for "Turn Windows features on or off" in the Start menu.
   - Check the box for Hyper-V and click OK.
   - Restart your computer if prompted.

2. **Install Docker Desktop**:
   - Download Docker Desktop from the [official Docker website](https://www.docker.com/products/docker-desktop).
   - Follow the installation instructions.

3. **Configuration**:
   - Once installed, open Docker Desktop and ensure it is running.
   - You can switch between Windows and Linux containers by right-clicking the Docker icon in the taskbar.

### For Windows 10 Home Users:
1. **Install WSL2 (Windows Subsystem for Linux)**:
   - Open PowerShell as Adm

 how are you


I'm here and ready to assist you! How can I help you today? If you have more questions about Docker or anything else, feel free to ask!



 what is philosoph


function_call: ResponseFunctionToolCall(arguments='{"query":"philosophy definition"}', call_id='call_AZ6n79xcdfWicZsMXJo8Q0qa', name='search', type='function_call', id='fc_687b456ec6a48192a65698c2d91e5cf0077f84ad0a6d4f14', status='completed')

It seems that the FAQ didn't provide a direct definition for "philosophy." However, I can explain it:

**Philosophy** is the study of fundamental questions about existence, knowledge, values, reason, mind, and language. It involves exploring concepts such as morality, reality, and human nature through critical analysis and systematic approaches. Philosophers seek to understand the nature of the world and our place within it, often asking profound questions like "What is truth?" or "What is the meaning of life?"

If you're looking for specific branches of philosophy or its applications, feel free to ask! What aspect of philosophy interests you the most?



 stop


## Alex full script 

In [85]:
!pip install markdown

In [82]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-19 07:17:32--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py.1’

chat_assistant.py.1 100%[===================>]   3.40K  --.-KB/s    in 0s      

2025-07-19 07:17:32 (17.6 MB/s) - ‘chat_assistant.py.1’ saved [3485/3485]



### Multiple Tools

In [89]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [91]:
add_entry_description = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

In [92]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(search, search_tool)

In [93]:
tools.add_tool(add_entry, add_entry_description)

In [94]:
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add an entry to the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'The question to be added to the FAQ database'},
    'answer': {'type': 'string', 'description': 'The answer to the question'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}}]

In [95]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [96]:
chat.run()

You: how do i do well in module 1


You: add fo FAQ entry


You: stop


Chat ended.


In [97]:
index.docs[-1]

{'question': 'How do I do well in Module 1?',
 'text': 'To excel in Module 1 (Docker and Terraform), follow these tips:\n1. **Understand the Basics**: Grasp fundamental Docker concepts like images and containers.\n2. **Follow Course Materials**: Engage with video lectures and documentation.\n3. **Hands-On Practice**: Set up a local environment and practice Docker commands.\n4. **Address Errors Promptly**: If encountering issues, such as ModuleNotFoundError for psycopg2, install necessary libraries with pip.\n5. **Join Discussion Forums**: Engage with classmates to ask questions and share knowledge.\n6. **Schedule Regular Study Time**: Dedicate consistent slots in your week for study.',
 'section': 'user added',
 'course': 'data-engineering-zoomcamp'}